In [1]:
# ===================================================================================
# UFC FIGHT OUTCOME PROBABILITY PREDICTION - FINAL PROJECT (SINGLE CELL)
# CIS 508 Machine Learning in Business
# Author: Anthony Nazaruk
# Objective: Predict P(Fighter A Wins) - probability between 0 and 1
# Models: Decision Tree, Logistic Regression, SVM, Neural Network, Naive Bayes,
#         Random Forest, XGBoost, KNN, Ensemble (Voting & Stacking)
# ===================================================================================

!pip install mlflow xgboost -q

import os, mlflow, mlflow.sklearn, pandas as pd, numpy as np, matplotlib.pyplot as plt, warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve, auc, brier_score_loss)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
print("Libraries imported!")

# --- DATABRICKS MLFLOW CONFIG ---
try:
    from google.colab import userdata
    os.environ["DATABRICKS_HOST"] = userdata.get("DATABRICKS_HOST")
    os.environ["DATABRICKS_TOKEN"] = userdata.get("DATABRICKS_TOKEN")
except: print("Configure Databricks secrets manually if needed.")
mlflow.set_tracking_uri("databricks")
experiment_name = "/Users/anazaruk@asu.edu/UFC_Fight_Probability_Prediction"
mlflow.set_experiment(experiment_name)
print(f"Experiment: {experiment_name}")

# --- LOAD DATASET ---
CSV_FILE = "ufc_fights_2019_2025_with_stats.csv"
TARGET = "fighter_a_won"
if not os.path.exists(CSV_FILE):
    from google.colab import files
    print(f"Please upload {CSV_FILE}"); uploaded = files.upload()
df = pd.read_csv(CSV_FILE)
print(f"Dataset: {df.shape[0]} rows, {df.shape[1]} cols | Target balance: {df[TARGET].mean():.2%} Fighter A wins")

# --- DATA PREPROCESSING ---
cols_to_drop = ['fight_key', 'fight_id', 'event_id', 'event_date', 'event_name', 'promotion',
                'fighter_a_id', 'fighter_a_name', 'fighter_b_id', 'fighter_b_name',
                'title_fight', 'fight_end_round', 'fight_result_type', 'fight_duration']
cols_to_drop = [c for c in cols_to_drop if c in df.columns]
df_model = df.drop(columns=cols_to_drop).copy()
cat_cols = [c for c in df_model.columns if c != TARGET and df_model[c].dtype == 'object']
num_cols = [c for c in df_model.columns if c != TARGET and df_model[c].dtype != 'object']
y = df_model[TARGET].copy()
X = df_model.drop(columns=[TARGET]).copy()
for col in cat_cols: X[col] = X[col].fillna('Unknown')
for col in num_cols: X[col] = X[col].fillna(X[col].median())
# *** FIX: Replace infinity and clip extreme values ***
X = X.replace([np.inf, -np.inf], np.nan)
for col in num_cols:
    X[col] = X[col].fillna(X[col].median())
    X[col] = X[col].clip(lower=-1e9, upper=1e9)
print(f"Features: {len(num_cols)} numerical, {len(cat_cols)} categorical")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Train: {X_train.shape[0]}, Test: {X_test.shape[0]}")

# --- PREPROCESSING PIPELINES ---
preprocess_tree = ColumnTransformer([("num", "passthrough", num_cols), ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)])
preprocess_scaled = ColumnTransformer([("num", StandardScaler(), num_cols), ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)])

# --- HELPER FUNCTIONS ---
def log_cm(cm, fname="cm.png"):
    fig, ax = plt.subplots(figsize=(4,4)); ax.imshow(cm, cmap="Blues")
    ax.set_xticks([0,1]); ax.set_xticklabels(["Loss","Win"]); ax.set_yticks([0,1]); ax.set_yticklabels(["Loss","Win"])
    for (i,j),v in np.ndenumerate(cm): ax.text(j,i,str(v),ha="center",va="center",fontsize=12)
    ax.set_xlabel("Predicted"); ax.set_ylabel("Actual"); plt.tight_layout(); fig.savefig(fname); plt.close(fig); mlflow.log_artifact(fname)

def log_curves(y_true, y_prob, prefix=""):
    fpr,tpr,_ = roc_curve(y_true,y_prob); fig,ax = plt.subplots(figsize=(4,4)); ax.plot(fpr,tpr); ax.plot([0,1],[0,1],'--',color='gray')
    ax.set_xlabel("FPR"); ax.set_ylabel("TPR"); ax.set_title(f"ROC AUC={auc(fpr,tpr):.3f}"); plt.tight_layout(); fig.savefig(f"{prefix}roc.png"); plt.close(fig); mlflow.log_artifact(f"{prefix}roc.png")
    prec,rec,_ = precision_recall_curve(y_true,y_prob); fig,ax = plt.subplots(figsize=(4,4)); ax.plot(rec,prec)
    ax.set_xlabel("Recall"); ax.set_ylabel("Precision"); ax.set_title(f"PR AUC={auc(rec,prec):.3f}"); plt.tight_layout(); fig.savefig(f"{prefix}pr.png"); plt.close(fig); mlflow.log_artifact(f"{prefix}pr.png")

def train_and_log(run_name, clf, preprocess, model_type, params):
    with mlflow.start_run(run_name=run_name):
        pipe = Pipeline([("prep", preprocess), ("clf", clf)])
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        y_prob = pipe.predict_proba(X_test)[:, 1]
        acc, prec, rec = accuracy_score(y_test,y_pred), precision_score(y_test,y_pred,zero_division=0), recall_score(y_test,y_pred,zero_division=0)
        f1, rocauc, brier = f1_score(y_test,y_pred,zero_division=0), roc_auc_score(y_test,y_prob), brier_score_loss(y_test,y_prob)
        mlflow.log_metric("test_accuracy",acc); mlflow.log_metric("test_precision",prec); mlflow.log_metric("test_recall",rec)
        mlflow.log_metric("test_f1",f1); mlflow.log_metric("roc_auc",rocauc); mlflow.log_metric("brier_score",brier)
        mlflow.log_param("model_type", model_type)
        for k,v in params.items(): mlflow.log_param(k,v)
        cm = confusion_matrix(y_test, y_pred, labels=[0,1]); log_cm(cm, f"{run_name}_cm.png"); log_curves(y_test, y_prob, f"{run_name}_")
        print(f"  {run_name}: Acc={acc:.4f}, F1={f1:.4f}, AUC={rocauc:.4f}, Brier={brier:.4f}")
        return {"run_name": run_name, "pipe": pipe, "f1": f1, "roc_auc": rocauc, "model_type": model_type}

mlflow.sklearn.autolog(log_models=True)
all_runs = []

# ===================================================================================
# 1) DECISION TREE (4 runs): max_depth x criterion
# ===================================================================================
print("\n" + "="*60 + "\n1) DECISION TREE (4 runs)\n" + "="*60)
dt_runs = []
for depth in [5, 15]:
    for crit in ["gini", "entropy"]:
        run = train_and_log(f"DT_d{depth}_{crit}", DecisionTreeClassifier(max_depth=depth, criterion=crit, random_state=42), preprocess_tree, "DecisionTree", {"max_depth":depth, "criterion":crit})
        dt_runs.append(run); all_runs.append(run)

# ===================================================================================
# 2) LOGISTIC REGRESSION (4 runs): C x penalty
# ===================================================================================
print("\n" + "="*60 + "\n2) LOGISTIC REGRESSION (4 runs)\n" + "="*60)
lr_runs = []
for C in [0.1, 1.0]:
    for pen in ["l1", "l2"]:
        run = train_and_log(f"LR_C{C}_{pen}", LogisticRegression(C=C, penalty=pen, solver="liblinear", max_iter=1000, random_state=42), preprocess_scaled, "LogisticRegression", {"C":C, "penalty":pen})
        lr_runs.append(run); all_runs.append(run)

# ===================================================================================
# 3) SVM (4 runs): kernel x C
# ===================================================================================
print("\n" + "="*60 + "\n3) SUPPORT VECTOR MACHINE (4 runs)\n" + "="*60)
svm_runs = []
for kern in ["linear", "rbf"]:
    for C in [0.1, 1.0]:
        run = train_and_log(f"SVM_{kern}_C{C}", SVC(kernel=kern, C=C, probability=True, random_state=42), preprocess_scaled, "SVM", {"kernel":kern, "C":C})
        svm_runs.append(run); all_runs.append(run)

# ===================================================================================
# 4) NEURAL NETWORK (6 runs): hidden_layers x learning_rate
# ===================================================================================
print("\n" + "="*60 + "\n4) NEURAL NETWORK (6 runs)\n" + "="*60)
nn_runs = []
for hidden in [(64,), (128,), (64,32)]:
    for lr in [0.001, 0.01]:
        run = train_and_log(f"NN_h{hidden}_lr{lr}", MLPClassifier(hidden_layer_sizes=hidden, learning_rate_init=lr, max_iter=500, early_stopping=True, random_state=42), preprocess_scaled, "NeuralNetwork", {"hidden_layers":str(hidden), "learning_rate":lr})
        nn_runs.append(run); all_runs.append(run)

# ===================================================================================
# 5) NAIVE BAYES (2 runs): var_smoothing
# ===================================================================================
print("\n" + "="*60 + "\n5) NAIVE BAYES (2 runs)\n" + "="*60)
nb_runs = []
for vs in [1e-9, 1e-7]:
    run = train_and_log(f"NB_vs{vs}", GaussianNB(var_smoothing=vs), preprocess_scaled, "NaiveBayes", {"var_smoothing":vs})
    nb_runs.append(run); all_runs.append(run)

# ===================================================================================
# 6) RANDOM FOREST (8 runs): n_estimators x max_depth x min_samples_split
# ===================================================================================
print("\n" + "="*60 + "\n6) RANDOM FOREST (8 runs)\n" + "="*60)
rf_runs = []
for n_est in [100, 200]:
    for depth in [10, 20]:
        for min_split in [2, 5]:
            run = train_and_log(f"RF_n{n_est}_d{depth}_s{min_split}", RandomForestClassifier(n_estimators=n_est, max_depth=depth, min_samples_split=min_split, random_state=42, n_jobs=-1), preprocess_tree, "RandomForest", {"n_estimators":n_est, "max_depth":depth, "min_samples_split":min_split})
            rf_runs.append(run); all_runs.append(run)

# ===================================================================================
# 7) XGBOOST (8 runs): n_estimators x learning_rate x max_depth
# ===================================================================================
print("\n" + "="*60 + "\n7) XGBOOST (8 runs)\n" + "="*60)
xgb_runs = []
for n_est in [100, 200]:
    for lr in [0.05, 0.1]:
        for depth in [4, 8]:
            run = train_and_log(f"XGB_n{n_est}_lr{lr}_d{depth}", XGBClassifier(n_estimators=n_est, learning_rate=lr, max_depth=depth, random_state=42, use_label_encoder=False, eval_metric='logloss', n_jobs=-1), preprocess_tree, "XGBoost", {"n_estimators":n_est, "learning_rate":lr, "max_depth":depth})
            xgb_runs.append(run); all_runs.append(run)

# ===================================================================================
# 8) KNN (8 runs): n_neighbors x weights x metric
# ===================================================================================
print("\n" + "="*60 + "\n8) K-NEAREST NEIGHBORS (8 runs)\n" + "="*60)
knn_runs = []
for k in [5, 11]:
    for w in ["uniform", "distance"]:
        for m in ["euclidean", "manhattan"]:
            run = train_and_log(f"KNN_k{k}_{w}_{m}", KNeighborsClassifier(n_neighbors=k, weights=w, metric=m, n_jobs=-1), preprocess_scaled, "KNN", {"n_neighbors":k, "weights":w, "metric":m})
            knn_runs.append(run); all_runs.append(run)

# ===================================================================================
# 9) ENSEMBLE - VOTING CLASSIFIER (1 run)
# ===================================================================================
print("\n" + "="*60 + "\n9a) ENSEMBLE - VOTING CLASSIFIER (1 run)\n" + "="*60)
with mlflow.start_run(run_name="Ensemble_Voting"):
    voting_clf = VotingClassifier(estimators=[
        ('rf', RandomForestClassifier(n_estimators=200, max_depth=20, random_state=42, n_jobs=-1)),
        ('xgb', XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=8, random_state=42, use_label_encoder=False, eval_metric='logloss')),
        ('lr', LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=1000, random_state=42)),
    ], voting='soft')
    pipe_vote = Pipeline([("prep", preprocess_scaled), ("clf", voting_clf)])
    pipe_vote.fit(X_train, y_train)
    y_pred_v = pipe_vote.predict(X_test); y_prob_v = pipe_vote.predict_proba(X_test)[:,1]
    acc_v, f1_v, auc_v, brier_v = accuracy_score(y_test,y_pred_v), f1_score(y_test,y_pred_v), roc_auc_score(y_test,y_prob_v), brier_score_loss(y_test,y_prob_v)
    mlflow.log_metric("test_accuracy",acc_v); mlflow.log_metric("test_f1",f1_v); mlflow.log_metric("roc_auc",auc_v); mlflow.log_metric("brier_score",brier_v)
    mlflow.log_param("model_type","VotingEnsemble"); mlflow.log_param("base_models","RF,XGB,LR")
    cm_v = confusion_matrix(y_test,y_pred_v,labels=[0,1]); log_cm(cm_v,"Ensemble_Voting_cm.png"); log_curves(y_test,y_prob_v,"Ensemble_Voting_")
    print(f"  Ensemble_Voting: Acc={acc_v:.4f}, F1={f1_v:.4f}, AUC={auc_v:.4f}, Brier={brier_v:.4f}")
    all_runs.append({"run_name":"Ensemble_Voting", "pipe":pipe_vote, "f1":f1_v, "roc_auc":auc_v, "model_type":"Ensemble"})

# ===================================================================================
# 9) ENSEMBLE - STACKING CLASSIFIER (1 run)
# ===================================================================================
print("\n" + "="*60 + "\n9b) ENSEMBLE - STACKING CLASSIFIER (1 run)\n" + "="*60)
with mlflow.start_run(run_name="Ensemble_Stacking"):
    stacking_clf = StackingClassifier(estimators=[
        ('rf', RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)),
        ('xgb', XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=8, random_state=42, use_label_encoder=False, eval_metric='logloss')),
        ('knn', KNeighborsClassifier(n_neighbors=11, weights='distance', metric='euclidean')),
    ], final_estimator=LogisticRegression(max_iter=1000, random_state=42), cv=5, n_jobs=-1)
    pipe_stack = Pipeline([("prep", preprocess_scaled), ("clf", stacking_clf)])
    pipe_stack.fit(X_train, y_train)
    y_pred_s = pipe_stack.predict(X_test); y_prob_s = pipe_stack.predict_proba(X_test)[:,1]
    acc_s, f1_s, auc_s, brier_s = accuracy_score(y_test,y_pred_s), f1_score(y_test,y_pred_s), roc_auc_score(y_test,y_prob_s), brier_score_loss(y_test,y_prob_s)
    mlflow.log_metric("test_accuracy",acc_s); mlflow.log_metric("test_f1",f1_s); mlflow.log_metric("roc_auc",auc_s); mlflow.log_metric("brier_score",brier_s)
    mlflow.log_param("model_type","StackingEnsemble"); mlflow.log_param("base_models","RF,XGB,KNN"); mlflow.log_param("meta_learner","LogisticRegression")
    cm_s = confusion_matrix(y_test,y_pred_s,labels=[0,1]); log_cm(cm_s,"Ensemble_Stacking_cm.png"); log_curves(y_test,y_prob_s,"Ensemble_Stacking_")
    print(f"  Ensemble_Stacking: Acc={acc_s:.4f}, F1={f1_s:.4f}, AUC={auc_s:.4f}, Brier={brier_s:.4f}")
    all_runs.append({"run_name":"Ensemble_Stacking", "pipe":pipe_stack, "f1":f1_s, "roc_auc":auc_s, "model_type":"Ensemble"})

# ===================================================================================
# FINAL SUMMARY
# ===================================================================================
print("\n" + "="*60)
print("FINAL SUMMARY")
print("="*60)
summary_df = pd.DataFrame(all_runs).sort_values(by="roc_auc", ascending=False)
print(f"\nTOTAL RUNS: {len(all_runs)}")
print(f"\nRuns by Model Type:")
print(f"  Decision Tree: {len(dt_runs)}, Logistic Regression: {len(lr_runs)}, SVM: {len(svm_runs)}")
print(f"  Neural Network: {len(nn_runs)}, Naive Bayes: {len(nb_runs)}, Random Forest: {len(rf_runs)}")
print(f"  XGBoost: {len(xgb_runs)}, KNN: {len(knn_runs)}, Ensemble: 2")
print(f"\nTop 10 Models by ROC-AUC:")
print(summary_df[["run_name", "model_type", "f1", "roc_auc"]].head(10).to_string(index=False))
best = summary_df.iloc[0]
print(f"\n🏆 BEST MODEL: {best['run_name']} ({best['model_type']}) | F1={best['f1']:.4f} | AUC={best['roc_auc']:.4f}")
print(f"\n✅ All {len(all_runs)} runs logged to Databricks MLflow: {experiment_name}")

# --- SAVE BEST MODEL ---
import pickle
best_pipe = best['pipe']
with open('ufc_fight_predictor.pkl', 'wb') as f: pickle.dump(best_pipe, f)
print(f"\n✅ Best model saved as: ufc_fight_predictor.pkl")

# --- SAMPLE PREDICTION ---
print("\n" + "="*60)
print("SAMPLE PREDICTION")
print("="*60)
sample = X_test.iloc[[0]]
prob = best_pipe.predict_proba(sample)[0][1]
print(f"Sample fight prediction:")
print(f"  P(Fighter A Wins): {prob:.2%}")
print(f"  P(Fighter B Wins): {1-prob:.2%}")
print(f"  Actual outcome: {'Fighter A Won' if y_test.iloc[0]==1 else 'Fighter B Won'}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.9/753.9 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.7 MB/s eta 0:00:00
Libraries imported!


2025/12/04 21:19:07 INFO mlflow.tracking.fluent: Experiment with name '/Users/anazaruk@asu.edu/UFC_Fight_Probability_Prediction' does not exist. Creating a new experiment.


Experiment: /Users/anazaruk@asu.edu/UFC_Fight_Probability_Prediction
Please upload ufc_fights_2019_2025_with_stats.csv


Saving ufc_fights_2019_2025_with_stats.csv to ufc_fights_2019_2025_with_stats.csv
Dataset: 7396 rows, 93 cols | Target balance: 50.00% Fighter A wins
Features: 72 numerical, 6 categorical
Train: 5916, Test: 1480

1) DECISION TREE (4 runs)
  DT_d5_gini: Acc=0.6243, F1=0.6160, AUC=0.6697, Brier=0.2289
🏃 View run DT_d5_gini at: https://dbc-4725f828-1619.cloud.databricks.com/ml/experiments/242438794216585/runs/750d0d591c60451f859a11ca760399c7
🧪 View experiment at: https://dbc-4725f828-1619.cloud.databricks.com/ml/experiments/242438794216585
  DT_d5_entropy: Acc=0.6203, F1=0.6151, AUC=0.6659, Brier=0.2300
🏃 View run DT_d5_entropy at: https://dbc-4725f828-1619.cloud.databricks.com/ml/experiments/242438794216585/runs/ca003644b2a3400794b0c777b2c16275
🧪 View experiment at: https://dbc-4725f828-1619.cloud.databricks.com/ml/experiments/242438794216585
  DT_d15_gini: Acc=0.5723, F1=0.5714, AUC=0.5587, Brier=0.3971
🏃 View run DT_d15_gini at: https://dbc-4725f828-1619.cloud.databricks.com/ml/experim

In [2]:
# ===================================================================================
# UFC FIGHT PREDICTION - HYPERPARAMETER FINE-TUNING (RUN AFTER INITIAL 46 RUNS)
# Based on best performers: XGBoost (depth=4) and Logistic Regression (C=0.1)
# This will add ~25 more fine-tuned runs
# ===================================================================================

print("="*60)
print("HYPERPARAMETER FINE-TUNING - PHASE 2")
print("Based on best initial results: XGBoost & Logistic Regression")
print("="*60)

# ===================================================================================
# 1) XGBOOST FINE-TUNING (12 runs)
# Best was: n_estimators=100, learning_rate=0.1, max_depth=4
# Fine-tune around those values + add regularization params
# ===================================================================================
print("\n" + "="*60)
print("1) XGBOOST FINE-TUNING (12 runs)")
print("="*60)

xgb_tuned_runs = []

# Fine-grained search around best params
xgb_fine_grid = {
    "n_estimators": [75, 100, 150],      # around 100
    "learning_rate": [0.08, 0.1, 0.12],  # around 0.1
    "max_depth": [3, 4, 5],              # around 4
    "reg_alpha": [0, 0.1],               # L1 regularization
    "reg_lambda": [1, 2],                # L2 regularization
    "subsample": [0.8, 1.0],             # row sampling
    "colsample_bytree": [0.8, 1.0],      # column sampling
}

# Run targeted combinations (not full grid - too many)
xgb_configs = [
    # Vary depth around best
    {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 3, "reg_alpha": 0, "reg_lambda": 1, "subsample": 1.0, "colsample_bytree": 1.0},
    {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 5, "reg_alpha": 0, "reg_lambda": 1, "subsample": 1.0, "colsample_bytree": 1.0},
    # Vary n_estimators
    {"n_estimators": 75, "learning_rate": 0.1, "max_depth": 4, "reg_alpha": 0, "reg_lambda": 1, "subsample": 1.0, "colsample_bytree": 1.0},
    {"n_estimators": 150, "learning_rate": 0.1, "max_depth": 4, "reg_alpha": 0, "reg_lambda": 1, "subsample": 1.0, "colsample_bytree": 1.0},
    # Vary learning rate
    {"n_estimators": 100, "learning_rate": 0.08, "max_depth": 4, "reg_alpha": 0, "reg_lambda": 1, "subsample": 1.0, "colsample_bytree": 1.0},
    {"n_estimators": 100, "learning_rate": 0.12, "max_depth": 4, "reg_alpha": 0, "reg_lambda": 1, "subsample": 1.0, "colsample_bytree": 1.0},
    # Add L1 regularization
    {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 4, "reg_alpha": 0.1, "reg_lambda": 1, "subsample": 1.0, "colsample_bytree": 1.0},
    {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 4, "reg_alpha": 0.5, "reg_lambda": 1, "subsample": 1.0, "colsample_bytree": 1.0},
    # Add L2 regularization
    {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 4, "reg_alpha": 0, "reg_lambda": 2, "subsample": 1.0, "colsample_bytree": 1.0},
    {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 4, "reg_alpha": 0, "reg_lambda": 5, "subsample": 1.0, "colsample_bytree": 1.0},
    # Add subsampling (reduces overfitting)
    {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 4, "reg_alpha": 0, "reg_lambda": 1, "subsample": 0.8, "colsample_bytree": 1.0},
    {"n_estimators": 100, "learning_rate": 0.1, "max_depth": 4, "reg_alpha": 0, "reg_lambda": 1, "subsample": 0.8, "colsample_bytree": 0.8},
]

for cfg in xgb_configs:
    run_name = f"XGB_TUNED_n{cfg['n_estimators']}_lr{cfg['learning_rate']}_d{cfg['max_depth']}_a{cfg['reg_alpha']}_l{cfg['reg_lambda']}_ss{cfg['subsample']}"
    with mlflow.start_run(run_name=run_name):
        clf = XGBClassifier(
            n_estimators=cfg['n_estimators'],
            learning_rate=cfg['learning_rate'],
            max_depth=cfg['max_depth'],
            reg_alpha=cfg['reg_alpha'],
            reg_lambda=cfg['reg_lambda'],
            subsample=cfg['subsample'],
            colsample_bytree=cfg['colsample_bytree'],
            random_state=42,
            use_label_encoder=False,
            eval_metric='logloss',
            n_jobs=-1
        )
        pipe = Pipeline([("prep", preprocess_tree), ("clf", clf)])
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        y_prob = pipe.predict_proba(X_test)[:, 1]

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        rocauc = roc_auc_score(y_test, y_prob)
        brier = brier_score_loss(y_test, y_prob)

        mlflow.log_metric("test_accuracy", acc)
        mlflow.log_metric("test_precision", prec)
        mlflow.log_metric("test_recall", rec)
        mlflow.log_metric("test_f1", f1)
        mlflow.log_metric("roc_auc", rocauc)
        mlflow.log_metric("brier_score", brier)
        mlflow.log_param("model_type", "XGBoost_Tuned")
        for k, v in cfg.items():
            mlflow.log_param(k, v)

        cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
        log_cm(cm, f"{run_name}_cm.png")
        log_curves(y_test, y_prob, f"{run_name}_")

        xgb_tuned_runs.append({"run_name": run_name, "pipe": pipe, "f1": f1, "roc_auc": rocauc, "brier": brier, "config": cfg})
        print(f"  {run_name}: AUC={rocauc:.4f}, Acc={acc:.4f}, F1={f1:.4f}, Brier={brier:.4f}")

print(f"\n✅ XGBoost Fine-Tuning: {len(xgb_tuned_runs)} runs completed")

# ===================================================================================
# 2) LOGISTIC REGRESSION FINE-TUNING (8 runs)
# Best was: C=0.1, penalty=l1
# Fine-tune C values and try elastic net
# ===================================================================================
print("\n" + "="*60)
print("2) LOGISTIC REGRESSION FINE-TUNING (8 runs)")
print("="*60)

lr_tuned_runs = []

lr_configs = [
    # Fine-tune C around 0.1
    {"C": 0.05, "penalty": "l1", "solver": "liblinear"},
    {"C": 0.08, "penalty": "l1", "solver": "liblinear"},
    {"C": 0.12, "penalty": "l1", "solver": "liblinear"},
    {"C": 0.15, "penalty": "l1", "solver": "liblinear"},
    # Fine-tune C with L2
    {"C": 0.05, "penalty": "l2", "solver": "liblinear"},
    {"C": 0.08, "penalty": "l2", "solver": "liblinear"},
    # Try elastic net (l1_ratio blends L1 and L2)
    {"C": 0.1, "penalty": "elasticnet", "solver": "saga", "l1_ratio": 0.5},
    {"C": 0.1, "penalty": "elasticnet", "solver": "saga", "l1_ratio": 0.7},
]

for cfg in lr_configs:
    run_name = f"LR_TUNED_C{cfg['C']}_{cfg['penalty']}"
    if 'l1_ratio' in cfg:
        run_name += f"_r{cfg['l1_ratio']}"

    with mlflow.start_run(run_name=run_name):
        if cfg['penalty'] == 'elasticnet':
            clf = LogisticRegression(C=cfg['C'], penalty=cfg['penalty'], solver=cfg['solver'],
                                     l1_ratio=cfg['l1_ratio'], max_iter=2000, random_state=42)
        else:
            clf = LogisticRegression(C=cfg['C'], penalty=cfg['penalty'], solver=cfg['solver'],
                                     max_iter=1000, random_state=42)

        pipe = Pipeline([("prep", preprocess_scaled), ("clf", clf)])
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        y_prob = pipe.predict_proba(X_test)[:, 1]

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        rocauc = roc_auc_score(y_test, y_prob)
        brier = brier_score_loss(y_test, y_prob)

        mlflow.log_metric("test_accuracy", acc)
        mlflow.log_metric("test_precision", prec)
        mlflow.log_metric("test_recall", rec)
        mlflow.log_metric("test_f1", f1)
        mlflow.log_metric("roc_auc", rocauc)
        mlflow.log_metric("brier_score", brier)
        mlflow.log_param("model_type", "LogisticRegression_Tuned")
        for k, v in cfg.items():
            mlflow.log_param(k, v)

        cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
        log_cm(cm, f"{run_name}_cm.png")
        log_curves(y_test, y_prob, f"{run_name}_")

        lr_tuned_runs.append({"run_name": run_name, "pipe": pipe, "f1": f1, "roc_auc": rocauc, "brier": brier, "config": cfg})
        print(f"  {run_name}: AUC={rocauc:.4f}, Acc={acc:.4f}, F1={f1:.4f}, Brier={brier:.4f}")

print(f"\n✅ Logistic Regression Fine-Tuning: {len(lr_tuned_runs)} runs completed")

# ===================================================================================
# 3) SVM FINE-TUNING (5 runs)
# Best was: linear kernel, C=0.1
# ===================================================================================
print("\n" + "="*60)
print("3) SVM FINE-TUNING (5 runs)")
print("="*60)

svm_tuned_runs = []

svm_configs = [
    {"kernel": "linear", "C": 0.05},
    {"kernel": "linear", "C": 0.08},
    {"kernel": "linear", "C": 0.15},
    {"kernel": "linear", "C": 0.2},
    {"kernel": "linear", "C": 0.5},
]

for cfg in svm_configs:
    run_name = f"SVM_TUNED_{cfg['kernel']}_C{cfg['C']}"
    with mlflow.start_run(run_name=run_name):
        clf = SVC(kernel=cfg['kernel'], C=cfg['C'], probability=True, random_state=42)
        pipe = Pipeline([("prep", preprocess_scaled), ("clf", clf)])
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        y_prob = pipe.predict_proba(X_test)[:, 1]

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        rocauc = roc_auc_score(y_test, y_prob)
        brier = brier_score_loss(y_test, y_prob)

        mlflow.log_metric("test_accuracy", acc)
        mlflow.log_metric("test_precision", prec)
        mlflow.log_metric("test_recall", rec)
        mlflow.log_metric("test_f1", f1)
        mlflow.log_metric("roc_auc", rocauc)
        mlflow.log_metric("brier_score", brier)
        mlflow.log_param("model_type", "SVM_Tuned")
        for k, v in cfg.items():
            mlflow.log_param(k, v)

        cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
        log_cm(cm, f"{run_name}_cm.png")
        log_curves(y_test, y_prob, f"{run_name}_")

        svm_tuned_runs.append({"run_name": run_name, "pipe": pipe, "f1": f1, "roc_auc": rocauc, "brier": brier, "config": cfg})
        print(f"  {run_name}: AUC={rocauc:.4f}, Acc={acc:.4f}, F1={f1:.4f}, Brier={brier:.4f}")

print(f"\n✅ SVM Fine-Tuning: {len(svm_tuned_runs)} runs completed")

# ===================================================================================
# TUNING SUMMARY
# ===================================================================================
print("\n" + "="*60)
print("FINE-TUNING SUMMARY")
print("="*60)

all_tuned = xgb_tuned_runs + lr_tuned_runs + svm_tuned_runs
tuned_df = pd.DataFrame(all_tuned).sort_values(by="roc_auc", ascending=False)

print(f"\nTotal tuned runs: {len(all_tuned)}")
print(f"\nTop 10 Tuned Models by ROC-AUC:")
print(tuned_df[["run_name", "roc_auc", "f1", "brier"]].head(10).to_string(index=False))

best_tuned = tuned_df.iloc[0]
print(f"\n🏆 BEST TUNED MODEL: {best_tuned['run_name']}")
print(f"   ROC-AUC: {best_tuned['roc_auc']:.4f}")
print(f"   F1: {best_tuned['f1']:.4f}")
print(f"   Brier: {best_tuned['brier']:.4f}")
print(f"   Config: {best_tuned['config']}")

# Compare to original best
print(f"\n📊 IMPROVEMENT CHECK:")
print(f"   Original best (XGB_n100_lr0.1_d4): ROC-AUC = 0.7168")
print(f"   New best ({best_tuned['run_name']}): ROC-AUC = {best_tuned['roc_auc']:.4f}")
improvement = (best_tuned['roc_auc'] - 0.7168) * 100
print(f"   Change: {improvement:+.2f}% points")

# Save best tuned model
import pickle
with open('ufc_fight_predictor_tuned.pkl', 'wb') as f:
    pickle.dump(best_tuned['pipe'], f)
print(f"\n✅ Best tuned model saved as: ufc_fight_predictor_tuned.pkl")

HYPERPARAMETER FINE-TUNING - PHASE 2
Based on best initial results: XGBoost & Logistic Regression

1) XGBOOST FINE-TUNING (12 runs)
  XGB_TUNED_n100_lr0.1_d3_a0_l1_ss1.0: AUC=0.7132, Acc=0.6554, F1=0.6605, Brier=0.2161
🏃 View run XGB_TUNED_n100_lr0.1_d3_a0_l1_ss1.0 at: https://dbc-4725f828-1619.cloud.databricks.com/ml/experiments/242438794216585/runs/91a168ef9be74cacb67f1725ee3d5a8b
🧪 View experiment at: https://dbc-4725f828-1619.cloud.databricks.com/ml/experiments/242438794216585
  XGB_TUNED_n100_lr0.1_d5_a0_l1_ss1.0: AUC=0.7085, Acc=0.6615, F1=0.6626, Brier=0.2179
🏃 View run XGB_TUNED_n100_lr0.1_d5_a0_l1_ss1.0 at: https://dbc-4725f828-1619.cloud.databricks.com/ml/experiments/242438794216585/runs/99cd9389c4354411b440c12d86d3d7e0
🧪 View experiment at: https://dbc-4725f828-1619.cloud.databricks.com/ml/experiments/242438794216585
  XGB_TUNED_n75_lr0.1_d4_a0_l1_ss1.0: AUC=0.7171, Acc=0.6615, F1=0.6644, Brier=0.2151
🏃 View run XGB_TUNED_n75_lr0.1_d4_a0_l1_ss1.0 at: https://dbc-4725f828-161